In [38]:
import cv2
from ultralytics.models import YOLO

Для этой работы я буду использовать данные которые нам скинули, там другая задача нету конвейера, ну зато поиграться можно

# Перенос в папки

In [4]:
import os

In [8]:
not_ = set([path[:len("ef8dbbf4")] for path in os.listdir("project-1-at-2026-01-11-09-36-8517fae7/labels")])

In [10]:
for_replace = []
for image in os.listdir("project-1-at-2026-01-11-09-36-8517fae7/images"):
    if image[:len("ef8dbbf4")] in not_:
        print(image)
        continue
    for_replace.append(image)


7b7ca22e-cube_medium-small_green_9.png
e0e22a67-sphere_blue_2.png
d08a401f-cube_small_blue_6.png
5f081f67-cube_medium_blue_4.png
ef8dbbf4-pyramid_red_7.png
3c3688f9-cube_large_red_4.png
6696a54c-cube_large_red_2.png
80e901b9-cube_large_blue_6.png
dd779550-cube_small_blue_9.png
70e96d90-pyramid_red_8.png
26a234d7-sphere_red_3.png
e61221b3-cylinder_blue_9.png
4658873f-pyramid_green_3.png
fb410372-cylinder_red_3.png
660f27ba-cylinder_red_9.png
023f18bb-cylinder_green_7.png
afd4e61d-cylinder_green_6.png
acd29e3d-sphere_blue_4.png
319ce950-pyramid_green_5.png


In [ ]:
for n in for_replace:
    os.replace(f"project-1-at-2026-01-11-09-36-8517fae7/images/{n}", f"project-1-at-2026-01-11-09-36-8517fae7/test/images/{n}")
    print(n)

# Обучение модели

In [21]:
model = YOLO("yolo11s.pt") # 11 потому что крутое описание на сайте


YOLO11 — это последняя итерация в серии Ultralytics YOLO детекторов объектов в реальном времени, переопределяющая возможности благодаря передовой точности, скорости и эффективности

[Ссылка](https://docs.ultralytics.com/ru/models/yolo11/)

In [20]:
#
epochs = 25
# model.train(
#     data="project-1-at-2026-01-11-09-36-8517fae7/model.yaml",
#     epochs=epochs,
#     batch=-1,
#     device="mps"
# )

New https://pypi.org/project/ultralytics/8.3.252 available 😃 Update with 'pip install -U ultralytics'


KeyError: 'model'

In [29]:
result = model.predict("project-1-at-2026-01-11-09-36-8517fae7/test/images/4658873f-pyramid_green_3.png", conf=.7)


image 1/1 /Users/egoglev/PycharmProjects/Robotics/first/module1/neuro_model/project-1-at-2026-01-11-09-36-8517fae7/test/images/4658873f-pyramid_green_3.png: 480x640 1 green_pyramid, 35.3ms
Speed: 1.3ms preprocess, 35.3ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)


In [73]:
import numpy as np

results = model.predict(image, conf=0.7)
result = results[0]



0: 480x640 1 green_pyramid, 36.2ms
Speed: 1.3ms preprocess, 36.2ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)
a tensor([2.]) a tensor([2.]) a [          2] a [[     563.01      261.18      706.55      398.78]] a [    0.93154]
green_pyramid [563 261 706 398]


In [71]:
model.names

{0: 'green_midi_cube',
 1: 'green_cylinder',
 2: 'green_pyramid',
 3: 'red_big_cube',
 4: 'red_pyramid',
 5: 'red_cylinder',
 6: 'red_sphere',
 7: 'blue_big_cube',
 8: 'blue_smal_cube',
 9: 'blue_midlle_cube',
 10: 'blue_cylinder',
 11: 'blue_sphere'}

In [48]:
class ModelWorker:
    def __init__(self, model_path, file_path, *, conf=.7):
        self.model = YOLO(model_path)
        self.conf = conf
        self.file_path = file_path
        # self.a = self.model.names

    def predict(self, img_path: str):
        img = cv2.imread(img_path)
        results = self.model.predict(img, conf=self.conf)
        result = results[0]
        for conf, cls, box in zip(
                        result.boxes.conf.cpu().numpy(),
                        result.boxes.cls.cpu().numpy(),
                        result.boxes.xyxy.cpu().numpy().astype(int)
                    ):

            x1, y1, x2, y2 = box
            cv2.rectangle(
                img,
                (x1, y1), (x2, y2),
                (0,255,0),
                2
            )

            cv2.putText(
                img,
                f"{model.names[int(cls)]} ({conf:.2f})",
                (x1, y1 - 10),
                cv2.FONT_HERSHEY_DUPLEX,
                .8,
                (0,0,255)
            )
            cv2.imwrite(self.file_path, img)


# Ставим точки для записи координат, полей

In [110]:
image = cv2.imread("project-1-at-2026-01-11-09-36-8517fae7/test/images/4658873f-pyramid_green_3.png")

for w in range(1, image.shape[0], int(image.shape[0] / 20)):
    for l in range(1, image.shape[0], int(image.shape[0] / 30)):
        cv2.rectangle(
            image,
            (w, l),
            (w+1, l+1),
            (0,0,0)
        )
        cv2.putText(
            image,

            f"{w}|{l}",
            (w, l - 1),
            cv2.FONT_HERSHEY_DUPLEX,
            .2,
            (0,0,255)
        )


In [111]:
cv2.imwrite("afsf.png", image)


True

In [120]:
from first.module1.utils.neuro_util import ModelWorker

In [116]:
mw = ModelWorker("", "runs/train6/weights/best.pt", file_path="tst_work.png")

In [117]:
mw.predict("project-1-at-2026-01-11-09-36-8517fae7/test/images/4658873f-pyramid_green_3.png")

AttributeError: 'ModelWorker' object has no attribute 'model'